In [1]:
import numpy as np
import graphinglib as gl
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation
from torch.utils.data import DataLoader
from string import ascii_lowercase
from copy import deepcopy

from projet.src.tools.smart_figure import SmartFigure
from projet.src.spectrums.spectrum import Spectrum
from projet.src.data_structures.spectrum_dataset import SpectrumDataset
from projet.src.spectrums.initial_guesses import find_peaks_gaussian_estimates
from projet.src.fitters.scipy_fitter import ScipyFitter
from projet.src.fitters.cnn_fitter import CNNFitter
from projet.src.fitters.res_net_fitter import ResNetFitter
from projet.src.fitters.score import *
from projet.src.tools.utilities import *

np.random.seed(0)

# Spectra

In [ ]:
np.random.seed(0)

filenames = ["single_gaussian", "distinct_gaussians", "distinct_twin_gaussians", "merged_twin_gaussians", 
             "pointy_gaussians", "contaminated_gaussians", "two_gaussian_components"]

spectra_filenames = []
for filename in filenames:
    for noise_level in ["smooth", "noisy", "very_noisy"]:
            spectra_filenames.append(f"projet/data/spectra/{filename}/{noise_level}.txt")

spectrums = [Spectrum.load(spectrum_file) for spectrum_file in spectra_filenames]

# Apply a small correction to the two_gaussians_components spectra to show the last two gaussians better
spectrums[-3].models[-2].mean = (69, 75)
spectrums[-3].models[-1].mean = (69, 75)
spectrums[-2].models[-2].mean = (69, 75)
spectrums[-2].models[-1].mean = (69, 75)
spectrums[-1].models[-2].mean = (69, 75)
spectrums[-1].models[-1].mean = (69, 75)
# --------------------------------------

figs = []
for i, (spectrum_type, spec_filenames) in enumerate(zip(
    np.array(spectrums, dtype=object).reshape(-1, 3), np.array(spectra_filenames, dtype=object).reshape(-1, 3)
)):
    figs.append(SmartFigure(
        num_rows=1,
        num_cols=3,
        title=f"{ascii_lowercase[i]}) {" ".join(spec_filenames[0].split('/')[-2].split('_'))}",
        remove_x_ticks=True,
        share_y=True,
        elements=[spectrum_type[0].plot, spectrum_type[1].plot, spectrum_type[2].plot],
        reference_labels=False,
        width_padding=-0.1,
        size=(12, 4),
    ))

fig_copies = deepcopy(figs)

In [29]:
fig_copies[0]._x_label = "Numéro du canal [-]"
fig_copies[0]._y_label = "Intensité [u. arb.]"
fig_copies[0]._remove_x_ticks = False
fig_copies[0]._title = "single gaussian"
fig_copies[0]._size = (11, 5)
# fig_copies[0].show()
fig_copies[0].save("projet/figures/presentation/spectra_0.png", dpi=500)

In [28]:
fig_copies[1]._title = "distinct gaussians"
fig_copies[4]._remove_x_ticks = False
fig_copies[4]._title = "pointy gaussians"
fig_copies[4]._y_lim = -4, 12.5
fig_copies[4]._yticks = [0,  5, 10]

fig_1_4 = SmartFigure(
    2, 1, "Numéro du canal [-]", "Intensité [u. arb.]", size=(11, 5), elements=[fig_copies[1], fig_copies[4]]
)
# fig_1_4.show()
fig_1_4.save("projet/figures/presentation/spectra_1_4.png", dpi=500)

In [ ]:
fig_copies[2]._title = "distinct twin gaussians"
fig_copies[2]._y_lim = -3, 11
fig_copies[2]._yticks = [0,  5, 10]
fig_copies[3]._title = "merged twin gaussians"
fig_copies[3]._remove_x_ticks = False

fig_2_3 = SmartFigure(
    2, 1, "Numéro du canal [-]", "Intensité [u. arb.]", size=(11, 5), elements=[fig_copies[2], fig_copies[3]]
)
# fig_2_3.show()
fig_2_3.save("projet/figures/presentation/spectra_2_3.png", dpi=500)

In [ ]:
fig_copies[5]._title = "contaminated gaussians"
# fig_copies[5]._y_lim = -3, 11
# fig_copies[5]._yticks = [0,  5, 10]
fig_copies[6]._title = "two gaussian components"
fig_copies[6]._remove_x_ticks = False

fig_5_6 = SmartFigure(
    2, 1, "Numéro du canal [-]", "Intensité [u. arb.]", size=(11, 5), elements=[fig_copies[5], fig_copies[6]]
)
# fig_5_6.show()
fig_5_6.save("projet/figures/presentation/spectra_5_6.png", dpi=500)